In [2]:
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px 


df = pd.read_csv('../Daten/cleaned_energy_data_fixed.xls', encoding='ISO-8859-1')


df.columns = df.columns.str.encode('latin1').str.decode('utf-8')


df.set_index("Year", inplace=True)


row_sums = df.sum(axis=1) 
df_percentage = df.div(row_sums, axis=0) * 100 


df_percentage = df_percentage * 2


df_percentage = df_percentage.clip(upper=100)


pivot_df = df.reset_index()


fig = go.Figure()

colors = px.colors.sequential.Viridis[:len(df.columns)]  


def add_traces(data, hover_suffix=""):
    fig.data = []
    for i, col in enumerate(data.columns):
        fig.add_trace(go.Scatter(
            x=data.index,
            y=data[col],
            fill='tonexty',  
            fillcolor=colors[i],
            name=col,
            hovertemplate=f'{col}: %{{y:.2f}}{hover_suffix}<extra></extra>', 
            line=dict(width=0)  
        ))


add_traces(df)


fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            x=0.15, 
            y=1.1,  
            showactive=True,
            buttons=[
                dict(
                    label="Alle",
                    method="relayout",
                    args=[{"xaxis.range": [df.index.min(), df.index.max()]}]
                ),
                dict(
                    label="Letztes 1 Jahr",
                    method="relayout",
                    args=[{"xaxis.range": [df.index.max() - 1, df.index.max()]}]
                ),
                dict(
                    label="Letzte 5 Jahre",
                    method="relayout",
                    args=[{"xaxis.range": [df.index.max() - 5, df.index.max()]}]
                ),
                dict(
                    label="Letzte 10 Jahre",
                    method="relayout",
                    args=[{"xaxis.range": [df.index.max() - 10, df.index.max()]}]
                ),
                dict(
                    label="Letzte 20 Jahre",
                    method="relayout",
                    args=[{"xaxis.range": [df.index.max() - 20, df.index.max()]}]
                ),
                dict(
                    label="Letzte 40 Jahre",
                    method="relayout",
                    args=[{"xaxis.range": [df.index.max() - 40, df.index.max()]}]
                )
            ]
        ),
        dict(
            type="buttons",
            direction="left",
            x=0.85,  
            y=1.15,  
            buttons=[
                dict(
                    label="Absolut",
                    method="update",
                    args=[
                        {"y": [df[col] for col in df.columns]},
                        {"yaxis": {"fixedrange": True}, "hovertemplate": "%{y:.2f}"}
                    ]
                ),
                dict(
                    label="Prozentual",
                    method="update",
                    args=[
                        {"y": [df_percentage[col] for col in df_percentage.columns]},
                        {"yaxis": {"title": "Prozent", "range": [0, 100], "fixedrange": True}, "hovertemplate": "%{y:.2f}%"}
                    ]
                )
            ],
            pad={"r": 10, "t": 10},
            showactive=True
        )
    ]
)


fig.update_layout(
    title="Endenergieverbrauch nach Energieträger", 
    title_x=0.5, 
    title_font=dict(size=16),
    xaxis=dict(
        title="Jahr", 
        showgrid=True, 
        gridwidth=2, 
        gridcolor='lightgray', 
        fixedrange=True
    ),
    yaxis=dict(
        title="Tausend Terajoules", 
        showgrid=True, 
        gridwidth=2, 
        gridcolor='lightgray', 
        fixedrange=True 
    ),
    plot_bgcolor='white', 
    margin=dict(l=40, r=40, t=40, b=40),
    legend=dict(
        title="Energieträger", 
        x=0.01, 
        y=0.9,  
        font=dict(size=10), 
        title_font=dict(size=12),
        bordercolor='rgba (0, 0, 0, 0.5)',
        borderwidth=1
    ),
    hovermode="x unified",
    template='plotly_white',  
)


fig.show()
